# **reForge + Zrok Modificacion Archi** 
*Basado en Sagemarker de WhiteZ*

**1. BORRA TODOS LOS ARCHIVOS, USALO CON CADA INSTALACIÓN NUEVA** (Puedes ejecutarlo en CPU)

In [ ]:
!rm -r /home/studio-lab-user/.cache
!rm -r /home/studio-lab-user/.conda
!rm -r /home/studio-lab-user/.jupyter
!rm -r /home/studio-lab-user/.sagemaker
!conda remove -n "env_name" --all
!rm -rf /home/studio-lab-user/*

**2. INSTALAR Y ACTUALIZAR PYTHON** (Puedes ejecutarlo en CPU)

In [ ]:
import os

os.environ["ROOT"] = "/home/studio-lab-user/stable-diffusion-webui-forge/"
os.environ["LD_PRELOAD"] = "/home/studio-lab-user/libtcmalloc_minimal.so.4"
os.environ['PATH'] = "./zrok-conection/bin:" + os.environ['PATH']

!sed -i 's/\r//' install.sh extensions.sh download.sh 

!bash install.sh
!zrok version

**DESCARGAR MODELOS EN CARPETA TMP DEBES DE EJECUTARLA CADA VEZ QUE INICIES UNA NUEVA SESIÓN** (*no funcionara con CPU, solo con GPU*)

In [ ]:
%cd /home/studio-lab-user
!bash download.sh

<h5><b>INSTALACION DE ZROK</b></h5>

##### *Usar solamente en la terminal para crear cuenta en zrok*
```bash
export PATH=$PATH:/tmp/zrok/bin
zrok invite
```

##### *activa o desactiva tu token de zrok, usar "enable" + tu token para activar y "disable" sin el token para desactivar*

In [ ]:
!zrok enable tu_token

**EJECUTAR SD**
SOLO EJECUTAR EN GPU

No olvide pegar su token NGROK a continuación. Crea un token aquí https://dashboard.ngrok.com/get-started/your-authtoken

<h5><b>LANZAR SD CON ZROK</b></h5>

In [ ]:
%cd ~/stable-diffusion-webui-reForge
from multiprocessing import Process
import time

def run_command1():
#   time.sleep(15)
    !zrok share public http://localhost:7860 --headless

def run_command2():
    time.sleep(2)
    !python launch.py --disable-xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue --no-download-sd-model --ckpt /home/studio-lab-user/stable-diffusion-webui-reForge/models/Stable-diffusion/tmp_models

if __name__ == '__main__':
    p1 = Process(target=run_command1)
    p2 = Process(target=run_command2)
    p1.start()
    p2.start()
    p1.join()
    p2.join()

In [ ]:
#Revisa el espacio disponible
!df -h | grep -E 'Avail|home'

In [ ]:
#ver todas las carpetas y su tamaño
!du -h --max-depth=1 /home/studio-lab-user/stable-diffusion-webui-reForge | sort -hr

In [ ]:
#borrar y limpiar carpeta outputs, (recomiendo hacerlo regularmente para no llenar el espacio en disco).
!rm -rf /home/studio-lab-user/stable-diffusion-webui-reForge/outputs/*

<h5><b>LANZAR SD CON NGROK</b></h5>

In [ ]:
#Don't reForget to paste your NGROK token here
%cd ~/stable-diffusion-webui-reForge
!python launch.py --ngrok tu_token_aqui --disable-xformers --no-half-vae --enable-insecure-extension-access --theme dark --gradio-queue --no-download-sd-model

<h4><b>🖼️ Step 5. Download all your outputs</b></h4>
If your setup folder doesn't have problem, skip this step.

In [ ]:
#download and compress all outputs
import os
import zipfile
def zip_folder(input_folder, output_folder, max_size_mb=20):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    zip_number = 1
    current_zip_size = 0
    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
    with zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED) as current_zip:
        for root, _, files in os.walk(input_folder):
            for file in files:
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                # Check if adding the current file exceeds the max size
                if current_zip_size + file_size > max_size_mb * 1024 * 1024:
                    # Close the current zip file
                    current_zip.close()
                    # Start a new zip file
                    zip_number += 1
                    current_zip_name = os.path.join(output_folder, f"part_{zip_number}.zip")
                    current_zip = zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED)
                    current_zip_size = 0
                # Add the file to the current zip
                current_zip.write(file_path, os.path.relpath(file_path, input_folder))
                current_zip_size += file_size
# Specify the input folder to be zipped
input_folder = 'stable-diffusion-webui-reForge/outputs/txt2img-images'  # Replace with the path to your input folder
# Specify the output folder where zip files will be saved
output_folder = 'outputs'  # Replace with the path to your output folder
# Specify the maximum size for each zip file in MB
max_size_mb = 199
# Create zip files with a maximum size of 20MB
zip_folder(input_folder, output_folder, max_size_mb)